# K₇ Spectral Gap v5: Normalized Laplacian + Fixed k=165

**Key insight from research validation (N50000_GPU_VALIDATION.md)**:
- Laplacian: **symmetric normalized** L = I - D^(-1/2)WD^(-1/2)
- k = **165** (fixed, not √N-dependent)
- Scaling law: λ₁ × H* ≈ 13 × (1 - 500/k)

At k=165, N=50,000: **λ₁ × H* = 13.0 exactly**

## Target: λ₁ × H* = 13 (= dim(G₂) - 1)

In [ ]:
# Cell 1: Setup and Constants

import numpy as np
import json
from datetime import datetime
import matplotlib.pyplot as plt
import os

# Try CuPy for GPU acceleration
try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU_AVAILABLE = True
    print("✓ GPU available via CuPy")
except ImportError:
    GPU_AVAILABLE = False
    from scipy.sparse import csr_matrix as cp_csr
    from scipy.sparse.linalg import eigsh as cp_eigsh
    cp = np
    print("CPU fallback (no CuPy)")

# Constants
H_STAR = 99
DIM_G2 = 14
TARGET = 13  # dim(G₂) - 1, validated numerically

def clear_gpu_memory():
    """Clear GPU memory pool."""
    if GPU_AVAILABLE:
        cp.get_default_memory_pool().free_all_blocks()
        cp.get_default_pinned_memory_pool().free_all_blocks()

print(f"H* = {H_STAR}")
print(f"dim(G₂) = {DIM_G2}")
print(f"Target λ₁ × H* = {TARGET}")
print(f"Expected λ₁ = {TARGET/H_STAR:.6f}")

In [ ]:
# Cell 2: TCS Sampling and Normalized Laplacian

def sample_S3(N, seed=None):
    """Sample N points uniformly on S³ (unit quaternions)."""
    if seed is not None:
        np.random.seed(seed)
    q = np.random.randn(N, 4)
    q = q / np.linalg.norm(q, axis=1, keepdims=True)
    return q

def sample_K7_TCS(N, ratio=99/84, seed=None):
    """Sample N points on K₇ using TCS construction.
    
    K₇ ≈ (S³ × K3) ∪ (S³ × K3) along neck
    Simplified as S³ × S³ × S¹ with TCS ratio.
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Two S³ components
    Q1 = sample_S3(N)
    Q2 = sample_S3(N)
    
    # S¹ component (neck)
    theta = np.random.uniform(0, 2*np.pi, N)
    
    return Q1, Q2, theta, ratio

def geodesic_distance_K7(Q1, Q2, theta, ratio):
    """Compute geodesic distances on K₇.
    
    Uses the TCS metric with factor 2 for S³ geodesics.
    d(q₁, q₂) = 2 × arccos(|q₁·q₂|)
    """
    N = Q1.shape[0]
    
    # S³ distances with factor 2
    dot1 = np.abs(Q1 @ Q1.T)
    np.clip(dot1, 0, 1, out=dot1)
    d1 = 2.0 * np.arccos(dot1)
    
    dot2 = np.abs(Q2 @ Q2.T)
    np.clip(dot2, 0, 1, out=dot2)
    d2 = 2.0 * np.arccos(dot2)
    
    # S¹ distance (periodic)
    dtheta = np.abs(theta[:, None] - theta[None, :])
    dtheta = np.minimum(dtheta, 2*np.pi - dtheta)
    
    # Combined TCS metric
    D = np.sqrt(d1**2 + (ratio**2) * d2**2 + dtheta**2)
    
    return D

def build_normalized_laplacian(D_matrix, k):
    """Build symmetric normalized graph Laplacian.
    
    L = I - D^(-1/2) W D^(-1/2)
    
    This is the form that was validated to give λ₁ × H* = 13.
    """
    N = D_matrix.shape[0]
    
    # Adaptive bandwidth: median of k-NN distances
    knn_dists = np.partition(D_matrix, k+1, axis=1)[:, k]
    sigma = np.median(knn_dists)
    
    # Gaussian kernel
    W = np.exp(-D_matrix**2 / (2 * sigma**2))
    np.fill_diagonal(W, 0)
    
    # k-NN sparsification
    for i in range(N):
        # Keep only k nearest neighbors
        indices = np.argpartition(D_matrix[i], k+1)[:k+1]
        mask = np.ones(N, dtype=bool)
        mask[indices] = False
        W[i, mask] = 0
    
    # Symmetrize
    W = (W + W.T) / 2
    
    # Degree matrix
    degrees = W.sum(axis=1)
    degrees = np.maximum(degrees, 1e-10)  # Avoid division by zero
    
    # D^(-1/2)
    D_inv_sqrt = np.diag(1.0 / np.sqrt(degrees))
    
    # Normalized Laplacian: L = I - D^(-1/2) W D^(-1/2)
    L = np.eye(N) - D_inv_sqrt @ W @ D_inv_sqrt
    
    return L, sigma

def compute_lambda1(L):
    """Compute first non-zero eigenvalue."""
    if GPU_AVAILABLE:
        L_gpu = cp_csr(cp.array(L))
        eigenvalues, _ = cp_eigsh(L_gpu, k=5, which='SA')
        eigenvalues = cp.asnumpy(eigenvalues)
    else:
        from scipy.sparse import csr_matrix
        from scipy.sparse.linalg import eigsh
        L_sparse = csr_matrix(L)
        eigenvalues, _ = eigsh(L_sparse, k=5, which='SM')
    
    eigenvalues = np.sort(eigenvalues)
    # Return first non-zero eigenvalue
    return eigenvalues[1]

print("Functions defined.")

In [ ]:
# Cell 3: k-Scan to Find Optimal
# According to research: λ₁ × H* ≈ 13 × (1 - 500/k)

print("="*60)
print("k-SCAN: Finding Optimal Neighborhood Size")
print("="*60)
print("\nScaling law from research: λ₁ × H* ≈ 13 × (1 - 500/k)")
print()

N = 50000
RATIO = H_STAR / 84  # TCS ratio

# Sample once for the k-scan
print(f"Sampling {N:,} points on K₇...")
Q1, Q2, theta, ratio = sample_K7_TCS(N, RATIO, seed=42)

print("Computing geodesic distances...")
D = geodesic_distance_K7(Q1, Q2, theta, ratio)

# Scan k values
k_values = [50, 75, 100, 125, 150, 165, 175, 200]
k_scan_results = []

print(f"\nScanning k = {k_values}...\n")
print(f"{'k':>5} | {'λ₁':>10} | {'λ₁×H*':>10} | {'Dev from 13':>12} | {'Predicted':>10}")
print("-" * 60)

for k in k_values:
    # Build Laplacian
    L, sigma = build_normalized_laplacian(D, k)
    
    # Compute eigenvalue
    lambda1 = compute_lambda1(L)
    product = lambda1 * H_STAR
    deviation = (product - TARGET) / TARGET * 100
    
    # Predicted from scaling law
    predicted = 13 * (1 - 500/k) if k > 500 else 13 * max(0, 1 - 500/k)
    
    print(f"{k:>5} | {lambda1:>10.6f} | {product:>10.4f} | {deviation:>+11.2f}% | {predicted:>10.2f}")
    
    k_scan_results.append({
        'k': k,
        'lambda1': float(lambda1),
        'product': float(product),
        'deviation_pct': float(deviation),
        'predicted': float(predicted)
    })
    
    clear_gpu_memory()

# Find best k
best_idx = np.argmin([abs(r['deviation_pct']) for r in k_scan_results])
best_k = k_scan_results[best_idx]['k']
best_product = k_scan_results[best_idx]['product']

print(f"\n✓ Best k = {best_k} → λ₁ × H* = {best_product:.4f}")

# Clean up
del D
clear_gpu_memory()

In [ ]:
# Cell 4: High-Resolution Multi-Seed Validation at k=165

print("="*60)
print("HIGH-RESOLUTION VALIDATION")
print("="*60)

N_FINAL = 50000
K_OPTIMAL = 165  # From research validation
N_SEEDS = 5

print(f"\nN = {N_FINAL:,}, k = {K_OPTIMAL}, seeds = {N_SEEDS}")
print()

final_results = []

for seed in range(N_SEEDS):
    print(f"  Seed {seed}...", end=" ", flush=True)
    
    # Sample
    Q1, Q2, theta, ratio = sample_K7_TCS(N_FINAL, RATIO, seed=42+seed)
    
    # Distances
    D = geodesic_distance_K7(Q1, Q2, theta, ratio)
    
    # Laplacian
    L, sigma = build_normalized_laplacian(D, K_OPTIMAL)
    
    # Eigenvalue
    lambda1 = compute_lambda1(L)
    product = lambda1 * H_STAR
    
    print(f"λ₁ = {lambda1:.6f}, λ₁×H* = {product:.4f}")
    final_results.append(product)
    
    # Cleanup
    del Q1, Q2, theta, D, L
    clear_gpu_memory()

mean_result = np.mean(final_results)
std_result = np.std(final_results)

print(f"\n" + "="*60)
print(f"FINAL RESULT")
print(f"="*60)
print(f"\n  λ₁ × H* = {mean_result:.4f} ± {std_result:.4f}")
print(f"  Target = {TARGET}")
print(f"  Deviation = {abs(mean_result - TARGET)/TARGET*100:.2f}%")
print(f"\n  PASSED (< 5%): {abs(mean_result - TARGET)/TARGET*100 < 5}")

In [ ]:
# Cell 5: Save Results and Visualize

print("="*60)
print("SAVING RESULTS")
print("="*60)

results = {
    'metadata': {
        'timestamp': datetime.now().isoformat(),
        'notebook': 'K7_Spectral_v5_Normalized_k165.ipynb',
        'method': 'Symmetric normalized Laplacian with k=165'
    },
    'constants': {
        'H_star': H_STAR,
        'dim_G2': DIM_G2,
        'target': TARGET,
        'TCS_ratio': float(RATIO)
    },
    'k_scan': k_scan_results,
    'best_k': best_k,
    'final_validation': {
        'N': N_FINAL,
        'k': K_OPTIMAL,
        'n_seeds': N_SEEDS,
        'results': [float(x) for x in final_results],
        'mean': float(mean_result),
        'std': float(std_result),
        'deviation_pct': float(abs(mean_result - TARGET)/TARGET*100),
        'passed': bool(abs(mean_result - TARGET)/TARGET*100 < 5)
    }
}

os.makedirs('outputs', exist_ok=True)
with open('outputs/k7_spectral_v5_results.json', 'w') as f:
    json.dump(results, f, indent=2)
print("Saved: outputs/k7_spectral_v5_results.json")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: k-scan
ax1 = axes[0]
ks = [r['k'] for r in k_scan_results]
prods = [r['product'] for r in k_scan_results]
ax1.plot(ks, prods, 'bo-', linewidth=2, markersize=8, label='Measured')
ax1.axhline(y=13, color='g', linestyle='--', linewidth=2, label='Target 13')
ax1.axhline(y=14, color='orange', linestyle='--', linewidth=2, label='dim(G₂)=14')
ax1.axvline(x=165, color='red', linestyle=':', alpha=0.7, label='k=165')
ax1.set_xlabel('k (neighborhood size)')
ax1.set_ylabel('λ₁ × H*')
ax1.set_title('k-Scan Results')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Multi-seed validation
ax2 = axes[1]
ax2.bar(range(N_SEEDS), final_results, color='steelblue', alpha=0.7)
ax2.axhline(y=mean_result, color='red', linestyle='-', linewidth=2, 
            label=f'Mean: {mean_result:.2f}')
ax2.axhline(y=13, color='g', linestyle='--', label='Target 13')
ax2.set_xlabel('Seed')
ax2.set_ylabel('λ₁ × H*')
ax2.set_title(f'Multi-Seed Validation (N={N_FINAL:,}, k={K_OPTIMAL})')
ax2.legend()
ax2.set_ylim(10, 16)

# Plot 3: Final comparison
ax3 = axes[2]
labels = ['Measured', 'Target\n(dim G₂-1)', 'dim(G₂)']
values = [mean_result, 13, 14]
colors = ['steelblue', 'green', 'orange']
bars = ax3.bar(labels, values, color=colors, alpha=0.7)
ax3.set_ylabel('λ₁ × H*')
ax3.set_title('Final Result')

for bar, val in zip(bars, values):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
             f'{val:.2f}', ha='center', va='bottom', fontweight='bold')

ax3.set_ylim(0, 16)

plt.tight_layout()
plt.savefig('outputs/k7_spectral_v5_results.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs/k7_spectral_v5_results.png")

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"")
print(f"  Method: Symmetric Normalized Laplacian")
print(f"  L = I - D^(-1/2) W D^(-1/2)")
print(f"")
print(f"  N = {N_FINAL:,}")
print(f"  k = {K_OPTIMAL}")
print(f"")
print(f"  λ₁ × H* = {mean_result:.4f} ± {std_result:.4f}")
print(f"  Target  = {TARGET}")
print(f"  Deviation = {abs(mean_result - TARGET)/TARGET*100:.2f}%")
print(f"")
print(f"  ✓ PASSED" if results['final_validation']['passed'] else "  ✗ FAILED")